# [Travelbrag](http://travelbrag.ca)
Andrea & Brett's travelogue.

In [121]:
import datetime
import pandas

cities = pandas.read_csv("cities.csv")
travelogue = pandas.read_csv("travelogue.csv", parse_dates=["arrived", "departed"])
today = datetime.date.today()
colours = {frozenset(["Andrea", "Brett"]): "#00FA21", frozenset(["Andrea"]): "#F2FA00",
           frozenset(["Brett"]): "#005CFA"}

# What was our last vacation?
A _vacation_ is classified as the last time Andrea and I travelled for non-work purposes together.

In [122]:
both_of_us = travelogue[travelogue["travellers"] == "Andrea & Brett"]
vacations = both_of_us[~both_of_us["work"]]
last_vacation = vacations.sort_values(by="departed").tail(1).iloc[0]
departed = last_vacation["departed"].date()

print("Our last vacation was to {}, {} for {} on {}.".format(last_vacation["city"],
                                                             last_vacation["country"],
                                                             last_vacation["purpose"],
                                                             departed.isoformat()))

Our last vacation was to Chiang Mai, THA for Mal's Wedding on 2020-02-23.


# Where have we been in the last 12 months?

In [123]:
import datetime

year_ago = pandas.Timestamp(today - datetime.timedelta(days=365))
travelogue[travelogue["arrived"] >= year_ago].sort_values(by="arrived")

,purpose,arrived,departed,travellers,city,country,work,comment
242,PyCon US,2019-04-30,2019-05-08,Brett,Cleveland,USA,True,NaN
243,DevDiv All-hands,2019-05-12,2019-05-16,Brett,Redmond,USA,True,NaN
244,BC Day in the States,2019-08-04,2019-08-05,Andrea & Brett,Everett,USA,False,NaN
245,Visit Ireland,2019-08-30,2019-09-03,Andrea & Brett,Dublin,IRL,False,NaN
246,Visit Ireland,2019-09-03,2019-09-05,Andrea & Brett,Tralee,IRL,False,NaN
247,Visit Ireland,2019-09-05,2019-09-08,Andrea & Brett,Galway,IRL,False,NaN
248,Core dev sprints 2019,2019-09-08,2019-09-14,Andrea & Brett,London,GBR,True,NaN
249,2020 Q2 All-hands,2019-10-15,2019-10-17,Brett,Redmond,USA,True,NaN
250,Visit Aunt Doris,2019-11-09,2019-11-11,Andrea & Brett,Victoria,CAN,False,NaN
251,Training during Snowageddon,2020-01-13,2020-01-16,Brett,Redmond,USA,True,NaN


# Where have we visited?

In [124]:
full_country_names = {
    "CAN": "Canada",
    "USA": "America",
    "GBR": "Great Britain",
    "JPN": "Japan",
    "BEL": "Belgium",
    "CZE": "Czech Republic",
    "ESP": "Spain",
    "FRA": "France",
    "CHE": "Switzerland",
    "CUB": "Cuba",
    "ARG": "Argentina",
    "DOM": "Dominican Republic",
    "ISL": "Iceland",
    "NLD": "Netherlands",
    "IND": "India",
    "NZL": "New Zealand",
    "IRL": "Ireland",
    "THA": "Thailand"
}

locations = travelogue[['city', 'country']].drop_duplicates()
countries = locations['country'].drop_duplicates().tolist()
city_names = tuple((row[0], row[1]) for row in locations.values.tolist())

print("(In chronological order of first visit)")
print()
print(len(cities), "cities across", len(countries), "countries:")
for country in countries:
    print("   ", full_country_names[country])
    for city, _ in filter(lambda x: x[1] == country, city_names):
        print("       ", city)

(In chronological order of first visit)

143 cities across 18 countries:
    America
        Yosemite Valley
        Las Vegas
        Anaheim
        Chicago
        Washington, D.C.
        Mammoth Lakes
        Montclair
        New York
        Atlantic City
        Addison
        Rosemont
        Seattle
        Pittsburgh
        Charlottesville
        Langley
        Meridian
        Roseville
        Atlanta
        Raleigh
        San Francisco
        Mountain View
        Monterey
        Portland
        Santa Clara
        Ann Arbor
        Redmond
        Austin
        Union City
        New Haven
        Boise
        Palo Alto
        Los Angeles
        Cleveland
        Everett
    Great Britain
        London
        Brighton
        York
        Edinburgh
        Oxford
        Ely
        Shoreham-by-Sea
        Manchester
        Liverpool
        Birmingham
        Lancaster
        Glasgow
        Stirling
        Bugbrooke
    Japan
        Annaka
        Hi

# Is the data valid?
Manually entering data into a spreadsheet is always error-prone, so some quick checks are always useful to catch common mistakes.

In [125]:
import datetime
import re
import urllib.parse

# CITIES
# Latitude
if any(lat < -90 or lat > 90 for lat in cities['latitude']):
    raise ValueError('malformed latitude')
    
# Longitude
if any(lng < -180 or lng > 180 for lng in cities['longitude']):
    raise ValueError('malformed longitude')

# TRAVELOGUE
# Travellers
if any(x not in {"Andrea", "Brett", "Andrea & Brett"} for x in travelogue['travellers']):
    raise ValueError('unrecognized travellers')
    
# City
if any(pandas.isnull(travelogue['city'])):
    raise ValueError('missing the city in the travelogue data')
elif not all(travelogue["city"].isin(cities["city"].values)):
    raise ValueError("city in travelogue but not in cities")

# Country
if any(len(x) != 3 or x.upper() != x for x in travelogue['country']):
    bad_countries = []
    raise ValueError('malformed country')
elif not all(travelogue["country"].isin(cities["country"].values)):
    raise ValueError("country in travelogue but not in cities")
elif len(pandas.merge(travelogue, cities, on=["city", "country"])) != len(travelogue):
    raise ValueError("city/country in travelogue not in cities data")
    
# Arrived/Departed
if any(x.arrived > x.departed for x in travelogue.itertuples()):
    raise ValueError("arrival date passed departure date")
        
print('All travelogue data is valid!')

All travelogue data is valid!


# GeoJSON map output
A GeoJSON file is generated to allow for easy mapping of visited cities.

In [126]:
import json

def split_travellers(travellers):
    return travellers.split(" & ")

complete_travelogue = pandas.merge(travelogue, cities, on=["city", "country"])
geo_data = {}
for trip in complete_travelogue.itertuples():
    location = trip.city, trip.country
    if location not in geo_data:
        data = {"coordinates": [trip.longitude, trip.latitude],
                "travellers": set(split_travellers(trip.travellers)), "last visit": trip.departed,
                "work": bool(trip.work)}
        geo_data[location] = data
    else:
        data = geo_data[location]
        data["travellers"].update(split_travellers(trip.travellers))
        if data["last visit"] < trip.departed:
            data["last visit"] = trip.departed
geojson = {"type": "FeatureCollection", "features": []}
for location, data in geo_data.items():
    point = {"type": "Feature", "geometry": {"type": "Point", "coordinates": data["coordinates"]},
             "properties": {"city": ", ".join(location), "last visit": data["last visit"].strftime("%Y-%m-%d"),
                            "marker-color": colours[frozenset(data["travellers"])], "work": data["work"]}}
    geojson["features"].append(point)
with open("travelogue.geojson", "w", encoding="utf-8") as file:
    json.dump(geojson, file, sort_keys=True)

In [127]:
import folium

world_map = folium.Map(location=[0, 0], zoom_start=3)
folium.GeoJson(geojson, name="Trips").add_to(world_map)
world_map